<a href="https://colab.research.google.com/github/rmshimomura/Workshop-TimeSeries/blob/master/Workshop_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pmdarima
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd
from pmdarima.arima import ndiffs, AutoARIMA, ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
drive.mount('/content/drive/')

In [ ]:
caminho_workshop = f''

In [ ]:
# Função para plotar séries mais facilmente
def plot(vals, title='', sec_plots=[], divisions=[], min_y=None, max_y=None):
  if min_y is None:
    min_y = min(vals) - np.std(vals)/5
  if max_y is None:
    max_y = max(vals) + np.std(vals)/5
  fig, ax = plt.subplots(figsize=(5,2), dpi=300)
  ax.set_title(title)
  plt.plot(vals, linewidth=0.5)
  for subplot in sec_plots:
    plt.plot(subplot, linewidth=0.5)
  plt.fill_between(
    [i for i in range(len(vals))],
    [max_y for i in range(len(vals))],
    [min_y for i in range(len(vals))],
    where=[True if i in divisions else False for i in range(len(vals))],
    color='grey', linestyle='--', alpha=0.5
  )

  ax.set_ylim(bottom=min_y, top=max_y)

  fig.tight_layout()
  plt.show()
  plt.close(fig)

## Trabalhando com séries reais

Vamos aplicar o que vimos até agora à uma série real. A série que usaremos de exemplo contém a quantidade de passagens em viagens internacionais por mês, os dados foram coletados durante 12 anos.

In [ ]:
serie = pd.read_csv(f'{caminho_workshop}/AirPassengers.csv')
serie.head()

Vamos ver o comportamento do nosso dado principal na coluna 'value'

In [ ]:
plot(serie['value'])

A série possui tendência? E sazonalidade? Ela é estacionária?

---

A série possuí uma **tendência**, e tanto a **média quanto a variância dos dados** muda ao longo da série, então podemos dizer que ela também é **não estacionária**. **Também há sazonalidade**, já que temos uma **repetição anual constante**. Podemos usar a função ndiffs para conferir a estacionariedade da série.


In [ ]:
ndiffs(np.log(serie['value']))

De acordo com a função, precisamos fazer uma diferenciação para obter uma série estacionária.

# Tratamento dos dados

## Retirando aumento da variância

Vamos aplicar a função `np.log` da biblioteca `numpy` para retirar a variação da série.

In [ ]:
serie_log = np.log(serie['value'])
plot(serie_log)

## Retirando a tendência

Agora que temos uma variância constante, precisamos retirar a tendência da série. Vamos tentar tirar a diferença de primeira ordem.

In [ ]:
# Definindo função de diferenciação
def lag_diff(serie, lag=1):
  return np.array([yt - ytlag for yt, ytlag in zip(serie[lag:], serie[:-lag])])

In [ ]:
serie_tratada = lag_diff(serie_log)
plot(serie_tratada)

Agora temos uma **série estacionária**, pois a média é constante em volta de 0 e sua variância não tem uma mudança significativa ao longo da série.

## Predição de séries temporais usando ARIMA

O modelo ARIMA (AutoRegressive Integrated Moving Average) é um modelo preditivo estatístico composto por três componentes

*   Autoregressão (AR), representa a quantidade de lags anteriores dos dados que nós usaremos no modelo
*   Integração (I), representa a quantidade de diferenciações necessárias para tornar a série estacionária
*   Média móvel (MA), representa a quantidade de lags anteriores dos **erros** que serão usados no modelo

Esses três componentes são representados pelas letras **p, i e q** respectivamente.

##Como definir p, i e q?

O valor da parte i do arima é a quantidade de **diferenciações feitas até se obter uam série estacionária**. No nosso caso, será 1

Em geral, modelos ARIMA usam apenas um dos parâmetros AR ou MA. Uma forma de definir qual deve ser usado e seu valor é através da autocorrelação e autocorrelação parcial.

### Autocorrelação e Autocorrelação parcial

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plot_acf(serie_tratada, zero=False)
_ = plot_pacf(serie_tratada, zero=False)

Para decidirmos qual dos parâmetros do ARIMA serão usados, devemos olhar para a autocorrelação de lag 1.

**Se ela for positiva, usaremos o parâmetro AR**, e seu valor será o **lag anterior ao primeiro lag não significativo da autocorrelação parcial**.

**Se ela for negativa, usaremos o parâmetro MA**, e seu valor será o **lag anterior ao primeiro lag não significativo da autocorrelação normal**.

No nosso caso, usaremos um modelo ARIMA de ordem (2,1,0)

In [ ]:
ordem = (2,1,0)
modelo = ARIMA(order=ordem)

Agora que definimos a ordem do nosso modelo, devemos encaixar seus parâmetros no nosso dado. Para fazermos predições mais tarde, **iremos separar nosso conjunto em treino e teste**.

In [ ]:
# 80% treino, 20% teste
divisao_treino_teste = int(len(serie) * 0.8)

treino = np.array(serie['value'][:divisao_treino_teste])
teste = np.array(serie['value'][divisao_treino_teste:])

Vamos treinar o modelo sob os dados de treino.

In [ ]:
modelo = modelo.fit(treino)
modelo.summary()

Com o modelo treinado, podemos fazer predições **sob o conjunto de treino**

In [ ]:
predicoes_treino = modelo.predict_in_sample()
plot(serie['value'], sec_plots=[predicoes_treino], min_y=0)

Vamos fazer predições para o resto da série e comparar com o conjunto de teste

In [ ]:
predicoes_teste = modelo.predict(len(teste))
plot(teste, sec_plots=[predicoes_teste])

Vamos juntar os dois gráficos para ter uma visão completa da performance do modelo

In [ ]:
plot(np.concatenate((treino, teste)), sec_plots=[np.concatenate((predicoes_treino, predicoes_teste))])

Por que nosso modelo ficou desse jeito durante o conjunto de teste?

## Usando SARIMA, o ARIMA sazonal

Cometemos um erro durante o passo de diferenciação. Qual foi esse erro?

In [ ]:
serie_tratada = lag_diff(serie_log)
plot(serie_tratada)

A série ainda possui sazonalidade. Nesse caso, o correto é fazer uma **diferenciação sazonal**.

A diferenciação sazonal é uma diferenciação de ordem k onde **k é o período da série**.

Vamos corrigir esse erro.

In [ ]:
serie_diff = lag_diff(serie_log, lag=12)
plot(serie_diff)

Após a diferenciação, podemos dizer que a série é estacionária?

---

Ela **não possui sazonalidade**, mas ainda possui uma média variada ao longo da série. Vamos aplicar a diferença de primeira ordem.

In [ ]:
serie_tratada2 = lag_diff(serie_diff, lag=1)
plot(serie_tratada2)

Agora temos uma série com média e variância constante. Mas como ficam os parâmetros do ARIMA?

Além dos parâmetros p, i e q, o modelo SARIMA tem mais **quatro parâmetros**, sendo eles **P, I, Q e M**. Com exceção do M, que representa o **período da série**, esses parâmetros são similares aos do ARIMA não sazonal, mas tem relação com a sazonalidade da série. 

##Definindo parâmetros P, I, Q e M

Vamos rever os gráficos de autocorrelação

In [ ]:
plot_acf(serie_tratada2, zero=False, lags=30)
_ = plot_pacf(serie_tratada2, zero=False, lags=30)

Agora temos um **valor negativo para a primeira autocorrelação**. Então a nossa ordem não sazonal será **(0,1,1)**.

A parte sazonal segue a mesma lógica, mas para a **autocorrelação na posição M**. Temos uma autocorrelação **negativa** na posição 12, então usaremos o parâmetro MA, e seu valor **será 1, pois a posição 1x12 é significativa, enquanto a posição 2x12 está bem próxima de não ser significativa**.

Logo, a ordem do nosso modelo SARIMA será (0,1,1)(0,1,1,12). Vamos criar o modelo e fazer as predições.

In [ ]:
ordem = (0,1,1)
ordem_sazonal = (0,1,1,12)
modelo_sazonal = ARIMA(order=ordem, seasonal_order=ordem_sazonal)
modelo_sazonal = modelo_sazonal.fit(treino)
modelo_sazonal.summary()

In [ ]:
modelo.summary()

In [ ]:
predicoes_treino = modelo_sazonal.predict_in_sample()
plot(serie['value'], sec_plots=[predicoes_treino], min_y=0)

In [ ]:
predicoes_teste = modelo_sazonal.predict(len(teste))
plot(teste, sec_plots=[predicoes_teste])

In [ ]:
plot(np.concatenate((treino, teste)), sec_plots=[np.concatenate((predicoes_treino, predicoes_teste))])

Agora nosso modelo segue muito mais fielmente à série original.

## AutoARIMA

Existe uma forma de encontrar os parâmetros **p, q, P e Q** do SARIMA automaticamente. Para isso, usaremos a função `AutoARIMA` da biblioteca `pmdarima`

In [ ]:
modelo_auto = AutoARIMA(d=1, D=1, m=12)
modelo_auto = modelo_auto.fit(treino)
modelo_auto.summary()

In [ ]:
predicoes_treino = modelo_auto.predict_in_sample()
plot(serie['value'], sec_plots=[predicoes_treino], min_y=0)
predicoes_teste = modelo_auto.predict(len(teste))
plot(teste, sec_plots=[predicoes_teste])
plot(np.concatenate((treino, teste)), sec_plots=[np.concatenate((predicoes_treino, predicoes_teste))])

O AutoARIMA faz a busca do melhor modelo através de uma busca sequencial dos melhores parâmetros afim de minimalizar a estatística AIC.